# Property-Based Testing with Hypothesis

The RSE team encourages everyone to write unit tests for their code. This gives you confidence that your code is correct and makes it easier to pin-point where bugs originate from if they do arise. However, it can often be quite time-consuming to write a sufficiently broad range of tests to cover all possible inputs. This post is going to illustrate how you can use a technique called "property-based testing" to address this. It will use Python and the [Hypothesis](https://hypothesis.readthedocs.io/en/latest/) library. However, similar libraries exist for a number of other languages.

**Note: This post was written using a [Jupyter notebook](https://jupyter.org/). You can [find it on GitHub](https://github.com/cmacmackin/property-based-testing) or [run it interactively on Binder](https://mybinder.org/v2/gh/cmacmackin/property-based-testing/HEAD?labpath=hypothesis-demo.ipynb). If you wish to run it on your own computer, you will need to install `hypothesis` and `pytest`.**

## Example Problem: Rational Numbers
Let's say we want to implement a class which can exactly represent rational numbers (i.e., numbers which are the quotient of two integers). In general, [floating point numbers](https://en.wikipedia.org/wiki/Floating-point_arithmetic) can only represent these approximately. We will use the [dataclass](https://docs.python.org/3/library/dataclasses.html) decorator to give us a number of "boilerplate" methods automatically (like the constructor, conversion to a string, and equality comparison). In Python, floating-point and integer values are immutable, so we will make our class immutable using the "frozen" argument in the dataclass [decorator](https://realpython.com/primer-on-python-decorators/). For the sake of brevity, the only operators we will implement for this class are addition and multiplication.

In [ ]:
from dataclasses import dataclass


@dataclass(frozen=True)
class Rational:
    """Representation of a rational number (i.e., a number which is the
    quotient of two integers).

    """
    numerator: int
    denominator: int

    def __add__(self, other: "int | Rational") -> "Rational":
        """Addition operator."""
        if isinstance(other, int):
            return Rational(self.numerator + self.denominator * other, self.denominator)
        return Rational(
            self.numerator * other.denominator + other.numerator * self.denominator,
            self.denominator * other.denominator
        )

    def __mul__(self, other: "int | Rational") -> "Rational":
        """Multiplication operator."""
        if isinstance(other, int):
            return Rational(self.numerator * other, self.denominator)
        return Rational(self.numerator * other.numerator, self.denominator * other.denominator)

    # In a real implementation, we'd also provide operators for subtraction, division,
    # raising to a power, comparisons, etc.
    
    def __float__(self) -> float:
        """Converts to a floating point number."""
        return self.numerator / self.denominator

Note the use of Python [type annotations](https://docs.python.org/3/library/typing.html) to indicate the expected data type of function arguments and results. This is not required for property-based testing but generally makes code more readable and easier to maintain. `int | Rational` indicates that either an integer or a `Rational` value is accepted.

Let's write a few simple tests for this class. We'll start off using conventional testing approaches. These tests are written to be run using [pytest](https://docs.pytest.org/en/7.4.x/).

In [ ]:
# To make `pytest` play nicely with Jupyter notebooks, we use the `ipytest`
# package. This provides the ipython magic command `%%ipytest` to run all
# the tests defined in a cell.
import ipytest
ipytest.autoconfig(addopts=("-W ignore::pytest.PytestAssertRewriteWarning", "-q", "--color=yes"))

In [ ]:
%%ipytest 

def test_rational_addition() -> None:
    assert Rational(1, 2) + Rational(1, 3) == Rational(5, 6)


def test_rational_multiplication() -> None:
    assert Rational(1, 2) * Rational(3, 4) == Rational(3, 8)


def test_rational_to_float() -> None:
    x = Rational(1, 2)
    assert float(x) == 0.5

However, this only tests a limited number of cases. What if instead we defined some property of the class and its methods which should hold true for *any* input?

## Introducing Hypothesis
The [Hypothesis](https://hypothesis.readthedocs.io/en/latest/) library provides "[strategies](https://hypothesis.readthedocs.io/en/latest/data.html)" for generating test data. Let's use these to write a test  to check that our implementation of addition of rational numbers is commutative. While commutivity is not enough on its own to prove that our implementation is correct (after all, it is also satisfied by multiplication), it can be [combined with other properties](https://fsharpforfunandprofit.com/posts/property-based-testing/#property-based-testing) to uniquely define addition.

In [ ]:
%%ipytest

from hypothesis import given
from hypothesis.strategies import integers

@given(integers(), integers(), integers(), integers())
def test_addition_commutative(num1: int, num2: int, denom1: int, denom2: int) -> None:
    r1 = Rational(num1, denom1)
    r2 = Rational(num2, denom2)
    assert r1 + r2 == r2 + r1

The `@given` decorator indicates that Hypothesis will generate a number of inputs to pass to the test as arguments. We use functions from Hypothesis to generate strategy objects to pass as arguments to `@given`. Here, we've passed in four strategies for generating integers, created with calls to [integers](https://hypothesis.readthedocs.io/en/latest/data.html#hypothesis.strategies.integers). This optionally accepts arguments specifying the minimum and maximum integer to generate, but we have chosen not to constrain them. The integers are then passed to the arguments named `num1`, `num2`, `denom1`, and `denom2`.  Hypothesis [integrates seamlessly](https://hypothesis.readthedocs.io/en/latest/supported.html#testing-frameworks) with both `pytest` and `unittest`. We run our tests exactly as we would normally with pytest. In a real project, this would probably mean executing `pytest` from the command-line, while here we use the `%%ipytest` magic.

When we run our tests, Hypothesis will actually execute `test_addition_commutative` a number of times, with different "examples" of integer inputs. These will include corner cases such as the maximum integer, minimum integer, and zero. If Hypothesis encounters an error or a test failing at any point, it will try to find simpler cases which reproduce that failure. It refers to this as "[shrinking](https://hypothesis.readthedocs.io/en/latest/data.html)" the data. How data shrinks will depend on the strategy. For integers, "examples . . . will shrink towards zero, and negative values will also shrink towards positive (i.e. -n may be replaced by +n)".

If you want to see more details of what Hypothesis is doing, you can play with some of its [command-line settings](https://hypothesis.readthedocs.io/en/latest/details.html#the-hypothesis-pytest-plugin). In particular, you can display some [statistics about your test](https://hypothesis.readthedocs.io/en/latest/details.html#statistics) or increase the [verbosity](https://hypothesis.readthedocs.io/en/latest/settings.html#verbose-output).

## Generating Custom Datatypes

It was rather tedious to have to ask for four integers and then manually create our rational numbers. Is there a way we can generate them directly? Yes, using the [builds](https://hypothesis.readthedocs.io/en/latest/data.html#hypothesis.strategies.builds) function. This takes as its first argument a class or function. It will then infer what strategies to use to provide arguments to the class/function based on [type annotations](https://docs.python.org/3/library/typing.html), if present. You can also specify these strategies manually with additional positional or keyword arguments to `builds`. Let's use this to implement a test for distributivity of addition and multiplication of rational numbers.

In [ ]:
%%ipytest

from hypothesis.strategies import builds

@given(builds(Rational), builds(Rational), builds(Rational))
def test_rational_distributivity(r1: Rational, r2: Rational, r3: Rational) -> None:
    assert r1 * (r2 + r3) == r1 * r2 + r1 * r3

This is a great example of the power of property-based testing. We have written a very short test defining a simple mathematical property which should hold true, but it has found a bug in our implementation which is not immediately obvious and which we may not have thought to test for on our own.

The exact nature of the failure being shown may vary depending on which version of Hypothesis you are using or, to some extent, by chance. The one seen at the time of writing this article occurred with the following inputs:
```
r1=Rational(numerator=0, denominator=-1)
r2=Rational(numerator=0, denominator=1)
r3=Rational(numerator=0, denominator=1)
```

The problem here is the handling of negative numbers in the numerator and denominator. Let's work out the two sides of our identity step-by-step:
```
LHS = 0/-1 * (0/1 + 0/1)
    = 0/-1 * (0*1 + 0*1)/(1*1)
    = 0/-1 * 0/1
    = (0*0)/(-1*1)
    = 0/-1

RHS = 0/-1 * 0/1 + 0/-1 * 0/1
    = (0*0)/(-1*1) + (0*0)/(-1*1)
    = 0/-1 + 0/-1
    = (0*-1 + 0*-1)/(-1*-1)
    = 0/1
```
Now, we know that `0/1 == 0/-1`, but the default [dataclass](https://docs.python.org/3/library/dataclasses.html#dataclasses.dataclass) equality operator does not. It is simply checking that the numerators of the LHS and RHS are identical and so are the denominators. Now we stop to think, we realise this could cause additional problems if we are comparing fractions like `1/2 == 2/4`. Let's reimplement our Rational class with a custom equality operator to fix this.

In [ ]:
@dataclass(frozen=True)
class Rational:
    """Representation of a rational number (i.e., a number which is the
    quotient of two integers).

    """
    numerator: int
    denominator: int

    def __add__(self, other: "int | Rational") -> "Rational":
        """Addition operator."""
        if isinstance(other, int):
            return Rational(self.numerator + self.denominator * other, self.denominator)
        return Rational(
            self.numerator * other.denominator + other.numerator * self.denominator,
            self.denominator * other.denominator
        )

    def __mul__(self, other: "int | Rational") -> "Rational":
        """Multiplication operator."""
        if isinstance(other, int):
            return Rational(self.numerator * other, self.denominator)
        return Rational(self.numerator * other.numerator, self.denominator * other.denominator)

    # In a real implementation, we'd also provide operators for subtraction, division,
    # raising to a power, comparisons, etc.
    
    def __float__(self) -> float:
        """Converts to a floating point number."""
        return self.numerator / self.denominator

    def __eq__(self, other: "int | Rational") -> None:
        return float(self) == float(other)

We try converting our rationals into floats and comparing these. Let's see if that works.

In [ ]:
ipytest.run("{MODULE}::test_rational_distributivity") # Reruns our distributivity test
# Note: this cell may be quite slow to finish

It turns out we've introduced a new bug. Because we are taking the quotient between the numerator and denominator, if the denominator is 0 then we get a `DivideByZero` error. A fraction with 0 in the denominator is undefined in any case, so let's adjust our test inputs to avoid having to deal with that. (A better approach might be to forbid the creation of rationals with a 0 denominator, but that would rob us of the chance to see how to modify Hypothesis strategies!)

## Defining our own Strategies

What we want is a way to tell Hypothesis to generate test data where we know the denominator is non-zero. We can still use the `builds` function to do this, but we'll need to give it an additional argument. We will also need to define a strategy for non-zero integers. We can use the [filter](https://hypothesis.readthedocs.io/en/latest/data.html#filtering) method to get rid of any integers with the value of zero. This method takes a Python callable object as an argument which will evaluate potential integer examples. The callable should return `True` if we want to use the integer and `False` if we do not. As it happens, the `bool()` function does exactly what we want, as `bool(0) == False` and `bool(n) == True`, where `n` is any non-zero integer.

In [ ]:
%%ipytest

non_zero_integers = integers().filter(bool)
finite_rationals = builds(Rational, denominator=non_zero_integers)

@given(finite_rationals, finite_rationals, finite_rationals)
def test_rational_distributivity(r1: Rational, r2: Rational, r3: Rational) -> None:
    assert r1 * (r2 + r3) == r1 * r2 + r1 * r3

Great, that's handled the divide-by-zero problem. Let's try writing a few more tests.

In [ ]:
%%ipytest

@given(non_zero_integers, non_zero_integers)
def test_multiply_by_reciprocal(x: int, y: int):
    r1 = Rational(x, y)
    r2 = Rational(y, x)
    assert r1 * r2 == 1

@given(finite_rationals, non_zero_integers)
def test_reducible_fractions(r: Rational, x: int) -> None:
    assert r == Rational(r.numerator * x, r.denominator * x)

@given(finite_rationals, finite_rationals)
def test_multiplication_commutative(r1: Rational, r2: Rational) -> None:
    assert r1 * r2 == r2 * r1

## Generalising our Tests
We wrote our addition and multiplication methods to be able to accept both rational and integer inputs (a so-called [union](https://docs.python.org/3/library/typing.html#typing.Union) type). However, so far we've only tested them with rationals. Using the [one_of](https://hypothesis.readthedocs.io/en/latest/data.html#hypothesis.strategies.one_of) strategy from Hypothesis, we can generate examples of both rationals and integers. `one_of` takes a number of other strategies as arguments and then will select examples from any of them.

In [ ]:
%%ipytest

from hypothesis.strategies import one_of

# Define a datatype that is the "union" of integers and rationals
Number = int | Rational
int_or_rational = one_of(integers(), finite_rationals)

@given(int_or_rational, int_or_rational)
def test_addition_commutative(x: Number, y: Number) -> None:
    assert x + y == y + x

@given(int_or_rational, int_or_rational)
def test_multiplication_commutative(x: Number, y: Number) -> None:
    assert x * y == y * x

@given(int_or_rational, int_or_rational, int_or_rational)
def test_rational_distributivity(r1: Number, r2: Number, r3: Number) -> None:
    assert r1 * (r2 + r3) == r1 * r2 + r1 * r3

This has revealed an oversight in our implementation. When the left-hand argument is a Rational, the Rational class can handle the arithmetic as expected. If both arguments are integers, the usual integer operators are called. However, we have not considered the case where the left-hand argument is an integer and the right-hand is rational. This case is not handled by `Rational.__add__` or `Rational.__mul__`. Fortunately, [Python supplies methods which can address this](https://docs.python.org/3/reference/datamodel.html#object.__radd__): `__radd__` and `__rmul__`. If one were to execute
```
r = Rational(1, 2)
x = 2 + r
```
then the second line will try to call
```
x = r.__radd__(2)
```

Let's re-implement our class with those methods and then try running our tests again.

In [ ]:
@dataclass(frozen=True)
class Rational:
    """Representation of a rational number (i.e., a number which is the
    quotient of two integers).

    """
    numerator: int
    denominator: int

    def __add__(self, other: "int | Rational") -> "Rational":
        """Addition operator."""
        if isinstance(other, int):
            return Rational(self.numerator + self.denominator * other, self.denominator)
        return Rational(
            self.numerator * other.denominator + other.numerator * self.denominator,
            self.denominator * other.denominator
        )

    def __mul__(self, other: "int | Rational") -> "Rational":
        """Multiplication operator."""
        if isinstance(other, int):
            return Rational(self.numerator * other, self.denominator)
        return Rational(self.numerator * other.numerator, self.denominator * other.denominator)

    def __radd__(self, other: int) -> "Rational":
        return self + other

    def __rmul__(self, other: int) -> "Rational":
        return self * other

    # In a real implementation, we'd also provide operators for subtraction, division,
    # raising to a power, comparisons, etc.
    
    def __float__(self) -> float:
        """Converts to a floating point number."""
        return self.numerator / self.denominator

    def __eq__(self, other: "int | Rational") -> None:
        return float(self) == float(other)

In [ ]:
%%ipytest

# We need to redefine finite_rationals to ensure it uses our new implementation of Rational
finite_rationals = builds(Rational, denominator=non_zero_integers)
int_or_rational = one_of(integers(), finite_rationals)

@given(int_or_rational, int_or_rational)
def test_addition_commutative(x: Number, y: Number) -> None:
    assert x + y == y + x

@given(int_or_rational, int_or_rational)
def test_multiplication_commutative(x: Number, y: Number) -> None:
    assert x * y == y * x

@given(int_or_rational, int_or_rational, int_or_rational)
def test_rational_distributivity(r1: Number, r2: Number, r3: Number) -> None:
    assert r1 * (r2 + r3) == r1 * r2 + r1 * r3

And that's fixed it. Our tests are passing!

## Caveats
We've seen how property-based testing can be useful in generating large quantities of diverse test data, allowing you to catch obscure bugs you wouldn't have considered. However, there are a few "gotchas" you need to be aware of when using it.

### Test run-time
Because Hypothesis runs your test many times, with many inputs, you need to be particularly conscious of how long your tests take. You should try to keep test data fairly small to help with this. By default, Hypothesis will complain if it [takes more than 200ms](https://hypothesis.readthedocs.io/en/latest/settings.html#hypothesis.settings.deadline) to run your test. This can sometimes be troublesome if developers are running tests on a number of different machines (e.g., Continuous Integration servers). A test that runs quickly on one machine may exceed that time-limit on another, slower machine. Sometimes it will be necessary to remove this deadline entirely, especially when running tests using Continuous Integration (CI). However, this could make your test-suite slow to execute, reducing the convenience of running it regularly. One way around this is to configure Hypothesis to [reduce the number of times a test is run](https://hypothesis.readthedocs.io/en/latest/settings.html#hypothesis.settings.max_examples) during development, but this increases the likelihood that a bug will be missed.

### Unexpected failures
Each time Hypothesis is run, it will try different combinations of inputs on your tests. It will keep a database of cases that caused failures in the past in a hidden folder and be sure to try those. However, it remains possible for tests that have particularly obscure bugs to suddenly fail without warning if Hypothesis happens to hit upon the right combination of inputs. This can be frustrating and can sometimes distract from whatever feature you are meaning to implement at the moment. That said, it is still finding a genuine bug and it's probably better to identify it this way than via a bug report.

### Non-deterministic functions
When Hypothesis finds a set of inputs that cause a test to fail, it will try running the test again to see if the same thing happens. If it does not, then Hypothesis will complain that the test is "flaky". If there is some non-deterministic element of the function you are testing then this could cause problems. However, you should generally try to avoid non-deterministic code anyway. If you need to generate random numbers (e.g., for a Monte-Carlo simulation), [Hypothesis can provide these for you](https://hypothesis.readthedocs.io/en/latest/data.html#hypothesis.strategies.randoms) in a way that it can play with nicely.

A related issue is with tests where the run-time could vary. Examples of this would be tests that need to wait for a network response or functions that employ caching to speed-up subsequent runs with the same input. If the first time a test is run it exceeds the time-limit but a second time it does not, then Hypothesis will also complain about the test being flaky.

### Hard-to-Define Data
In our examples above we are dealing with a simple data structure which is valid for any combination of integers. (Or almost any; we saw how having a denominator of 0 can cause problems.) The more complicated a data structure, the greater the chance it can end up in an invalid state, with attributes holding values that are in some way contradictory. This makes it more difficult to [devise strategies which produce useful test data](https://hypothesis.works/articles/generating-the-right-data/). With scientific software, the problem can become particularly severe, as we are often dealing with arrays of data which need to look a certain way, but which you can't easily define in terms of mathematical functions. For example, the function you are testing might need some data from a JET pulse. You can not just pass it an array of random floats! You will somehow need to generate some data that looks realistic and is self-consistent. This can be difficult and sometimes impossible. In these situations, you may have to give up on property-based testing and just resort to using a few real-world datasets as inputs.

### Numerical error
In scientific programming, we are often employing floating-point arithmetic. Unfortunately, due to the finite-precision with which floating-point numbers can be stored, there will always be some "round-off error" in the results. Furthermore, the numerical methods we use to solve mathematical problems will normally have some "truncation error" due to the approximations involved. As a result, we are used to testing for equality within some tolerance.

The problem with property-based testing is that it is very good at finding particularly poorly-behaved inputs. For numerical methods, it can very often find an obscure case that will have quite high error, making it difficult to select an appropriate tolerance. This makes it hard to write good property-based tests for numerical methods. Simple input data (e.g., linear functions) can sometimes work well, but more complicated inputs often do not. Generally, I would advise only to use property-based testing on numerical methods if you have a solid understanding of how the truncation error and round-off error are expected to behave. Unless there is a mathematical proof that the error will not exceed some tolerance, odds are that Hypothesis will find a case where it does!

## Conclusion
This was a simple introduction to the power of property-based testing. It gives you a bit of understanding of what it can achieve and how you can use it in Python. It also warns against some of the pitfalls of this approach (long runtimes and unexpected failures) and when it might not be appropriate (working with complicated data or numerical methods). However, this post is only scratching the surface of what Hypothesis can do and the different sorts of properties you can test for.

Despite the caveats outlined in the previous section, property-based testing is an incredibly useful technique with many potential applications for scientific programmers. Take a look at some of the links below to learn more about it and start thinking about how to apply it to your own code!

## Further Reading
- [Hypothesis Documentation](https://hypothesis.readthedocs.io/en/latest/index.html)
- [Hypothesis Quick-Start Guide](https://hypothesis.readthedocs.io/en/latest/quickstart.html)
- [Choosing Properties for Property-Based Testing](https://fsharpforfunandprofit.com/posts/property-based-testing-2/) (examples in F#, but concepts apply to all languages)
- [Hypothesis for the Scientific Stack](https://hypothesis.readthedocs.io/en/latest/numpy.html)
- [Property-based testing libraries for other languages](https://hypothesis.works/articles/quickcheck-in-every-language/)
- [Hypothesis Blog](https://hypothesis.works/articles/)